In [1]:
import os
import keras as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file = 'dataset_1'

train_datagen = K.preprocessing.image.ImageDataGenerator()

validation_datagen = K.preprocessing.image.ImageDataGenerator()

test_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(file, 'train'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        )

validation_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'validation'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

test_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'test'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

Found 12556 images belonging to 28 classes.
Found 1710 images belonging to 28 classes.
Found 3515 images belonging to 28 classes.


In [3]:
early_stop = K.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='max')

In [40]:
class LeNetRelu:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        
        model.add(Conv2D(7, kernel_size=3, strides=2, input_shape=input_shape))
        model.add(Activation('selu'))
#         model.add(LeakyReLU())
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(14, kernel_size=3, strides=2, border_mode="same"))
#         model.add(Activation('selu'))
        model.add(LeakyReLU())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation('selu'))
#         model.add(LeakyReLU())
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        model.summary()

        model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])
        return model
    
def f_score_calc(model):
    X_test = []
    y_test = []
    y_pred = []
    for i in range(len(test_generator)):
        X_ex, y_ex = test_generator.next()
        X_test.append(X_ex)
        y_test.append(y_ex.argmax(axis=-1))
        y_pred.append(model.predict(X_ex).argmax(axis=-1))

    val_f1_score = f1_score(y_test, y_pred, average='micro')
    return val_f1_score

In [5]:
INPUT_SHAPE = (50, 50, 1)
NB_EPOCH = 50
BATCH_SIZE = 32
VERBOSE = 1
OPTIMIZER = Adam()
CLASSES = 28
STEPS = (int)(train_generator.n/BATCH_SIZE)

In [6]:
# K.backend.clear_session()

In [7]:
# K.backend.get_session()

In [41]:
# K.backend.get_session()

model = LeNetRelu.build(input_shape=INPUT_SHAPE, classes=CLASSES)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS,
        epochs=NB_EPOCH,
#         verbose=VERBOSE,
        validation_data=validation_generator,
#         validation_steps=STEPS,
        callbacks = [early_stop]
)

print("\nF1 Score: {}".format(f_score_calc(model)))

# K.backend.clear_session()
del model


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(14, kernel_size=3, strides=2, padding="same")`
  # This is added back by InteractiveShellApp.init_path()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 24, 24, 7)         70        
_________________________________________________________________
activation_26 (Activation)   (None, 24, 24, 7)         0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 12, 12, 7)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 6, 6, 14)          896       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 6, 6, 14)          0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 3, 3, 14)          0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 126)               0         
__________

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('c1_acc.jpg')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('c1_loss.jpg')
plt.show()